# 大使模式

## 目标

将调用远程服务时的一些通用功能从调用者业务代码中解耦合出来，独立成一个代理模块，由于通常应用在远程调用当中，因此像远程服务的大使一样，称为大使模式。

在微服务框架中，大使模式是使用非常频繁的。

![大使模式](pic/ambassador.png)

## 应用场景

- 远程调用的权限控制
- 实现远程调用日志功能
- 实现微服务熔断
- 远程服务负载限流
- 自适应网络连接

In [1]:
%maven ch.qos.logback:logback-classic:1.2.3
%maven org.slf4j:slf4j-api:1.7.25
%maven org.apache.commons:commons-lang3:3.7

In [2]:
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;
import org.apache.commons.lang3.RandomUtils;

In [3]:
// 远程服务接口及实现
public interface RemoteService {
    int remoteAdd(int x, int y) throws IOException;
}

// 远程服务实现成单例模式
public class RemoteServiceImpl implements RemoteService {
    private final Logger LOGGER = LoggerFactory.getLogger(getClass());
    private static RemoteService service = null;
    private RemoteServiceImpl() {}
    
    public static synchronized RemoteService getInstance() {
        if(service == null) {
            service = new RemoteServiceImpl();
        }
        return service;
    }
    
    // 模拟时延及发生错误的情况，发生概率约1/3，不发生错误时返回x，y之和
    @Override
    public int remoteAdd(int x, int y) throws IOException{
        try {
            // 每次调用随记模拟50ms - 500ms的时延
            Thread.sleep(RandomUtils.nextLong(50, 500));
        } catch (InterruptedException e) {
            // pass
        }
        int state = RandomUtils.nextInt(0,3);
        if(state == 0) {
            LOGGER.error("模拟远程调用发生错误");
            throw new IOException("远程服务发生错误");
        } 
        return x+y;
    }
}

In [4]:
// 客户端调用
public class RemoteServiceClient {
    private final Logger LOGGER = LoggerFactory.getLogger(getClass());
    private RemoteService remote;
    private int x, y;
    public RemoteServiceClient(RemoteService remote, int x, int y) {
        this.remote = remote;
        this.x = x;
        this.y = y;
    }
    public void setRemote(RemoteService remote) { this.remote = remote; }
    public int getX() { return x; }
    public void setX(int x) { this.x = x; }
    public int getY() { return y; }
    public void setY(int y) { this.y = y; }
    
    public int sum() throws IllegalArgumentException, IOException{
        if(remote == null) {
            LOGGER.error("远程服务对象不能为空");
            throw new IllegalArgumentException("远程服务对象不能为空");
        }
        return remote.remoteAdd(x, y);
    }
}

In [5]:
// 大使模式实现类，实现错误重试，调用时间，日志记录通用功能
public class RemoteServiceAmbassador implements RemoteService {
    private final Logger LOGGER = LoggerFactory.getLogger(getClass());
    // 大使对象中包含真正的远程服务对象
    private RemoteService service = RemoteServiceImpl.getInstance();
    private static final int RETRIES = 3;
    
    @Override
    public int remoteAdd(int x, int y) throws IOException {
        long start = System.currentTimeMillis();
        LOGGER.info("开始调用远程服务: x={}, y={}", x, y);
        int tryTimes = 0;
        boolean success = false;
        int result = 0;
        while(tryTimes < RETRIES) {
            try {
                result = service.remoteAdd(x, y);
                success = true;
                break;
            } catch (IOException e) {
                tryTimes ++;
                LOGGER.error("请求错误: {}, 进行第 {} 次重试", e.getMessage(), tryTimes);
            }
        }
        LOGGER.info("远程服务调用时间: {}ms", System.currentTimeMillis()-start);
        if(!success) {
            throw new IOException("远程服务调用失败");
        }
        LOGGER.info("远程服务返回结果: {}", result);
        return result;
    }
}

In [6]:
// 测试主类
public class App {
    public static void main(String[] args) throws IOException {
        // 无大使模式时的调用方式
        RemoteServiceClient client = new RemoteServiceClient(RemoteServiceImpl.getInstance(),
            100, 20);
        System.out.println(String.format("%d + %d = %d", client.getX(), client.getY(), 
            client.sum()));
        // 使用大使模式的调用方式
        client.setRemote(new RemoteServiceAmbassador());
        client.setX(123);
        client.setY(456);
        System.out.println(String.format("%d + %d = %d", client.getX(), client.getY(), 
            client.sum()));
    }
}

In [10]:
// 每次调用都会有不同的结果，下面是尝试多次后留下的集中典型结果
App app = new App()

In [11]:
// 两次调用都立刻成功返回，使用大使模式调用时得到了服务调用时间和调用日志
app.main(null)

100 + 20 = 120


[IJava-executor-0] INFO REPL.$JShell$31$RemoteServiceAmbassador - 开始调用远程服务: x=123, y=456
[IJava-executor-0] INFO REPL.$JShell$31$RemoteServiceAmbassador - 远程服务调用时间: 226ms
[IJava-executor-0] INFO REPL.$JShell$31$RemoteServiceAmbassador - 远程服务返回结果: 579


123 + 456 = 579


In [13]:
// 第一次调用时远程服务发生错误，主线程直接终止，第二次调用没有发生
app.main(null)

[IJava-executor-0] ERROR REPL.$JShell$29$RemoteServiceImpl - 模拟远程调用发生错误


EvalException: 远程服务发生错误

In [15]:
// 第一次调用时成功返回，第二次调用时发生了两次错误，大使模式帮助进行了两次重试后获得结果
app.main(null)

100 + 20 = 120


[IJava-executor-0] INFO REPL.$JShell$31$RemoteServiceAmbassador - 开始调用远程服务: x=123, y=456
[IJava-executor-0] ERROR REPL.$JShell$29$RemoteServiceImpl - 模拟远程调用发生错误
[IJava-executor-0] ERROR REPL.$JShell$31$RemoteServiceAmbassador - 请求错误: 远程服务发生错误, 进行第 1 次重试
[IJava-executor-0] ERROR REPL.$JShell$29$RemoteServiceImpl - 模拟远程调用发生错误
[IJava-executor-0] ERROR REPL.$JShell$31$RemoteServiceAmbassador - 请求错误: 远程服务发生错误, 进行第 2 次重试
[IJava-executor-0] INFO REPL.$JShell$31$RemoteServiceAmbassador - 远程服务调用时间: 573ms
[IJava-executor-0] INFO REPL.$JShell$31$RemoteServiceAmbassador - 远程服务返回结果: 579


123 + 456 = 579
